##### Import libraries, function declaration and .csv load

In [1]:
import pandas as pd
import json

In [2]:
def jsonprints(x):
    print(json.dumps(json.loads(x), indent=2))

In [3]:
# recursive flatten function from https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [4]:
def nanPercentage (x, col):
    nulls = sum(pd.isnull(x[col]))
    noNulls = x[x[col].isna() == False][col].count()
    total = nulls + noNulls
    if total == x.shape[0]:
        return nulls/total
    else:
        print('Something went wrong!')
        return -1

In [5]:
data = pd.read_csv("../data/train.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##### Select the column of interest, parse its json into several dataframe columns

In [6]:
df = data
# df = data.sample(n=5000)
# df.describe(include='all')

traffic = df['trafficSource']

In [7]:
trafficFlatten = [flatten_json(json.loads(d)) for d in df['trafficSource']]
t = pd.DataFrame(trafficFlatten)

In [8]:
t.head()

,adContent,adwordsClickInfo_adNetworkType,adwordsClickInfo_criteriaParameters,adwordsClickInfo_gclId,adwordsClickInfo_isVideoAd,adwordsClickInfo_page,adwordsClickInfo_slot,campaign,campaignCode,isTrueDirect,keyword,medium,referralPath,source
0,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


##### General idea of the columns content

In [9]:
for col in t:
    print(col + ':', nanPercentage(t, col))
    
# will add here all the columns that are not worth to keep for droping all of the together
dropList = []

adContent: 0.9878869433289106
adwordsClickInfo_adNetworkType: 0.9762519462669852
adwordsClickInfo_criteriaParameters: 0.0
adwordsClickInfo_gclId: 0.9761401777009538
adwordsClickInfo_isVideoAd: 0.9762519462669852
adwordsClickInfo_page: 0.9762519462669852
adwordsClickInfo_slot: 0.9762519462669852
campaign: 0.0
campaignCode: 0.9999988933805344
isTrueDirect: 0.6967807333124552
keyword: 0.5565510212437739
medium: 0.0
referralPath: 0.6337742474157669
source: 0.0


##### ```trafficSource['adContent']```

¿? Change this to a bool var "adHasContent"

In [10]:
t['adContent'].value_counts()

# t['adHasContent'] = 
# drop.append('adContent')

Google Merchandise Collection                  5122
Google Online Store                            1245
Display Ad created 3/11/14                      967
Full auto ad IMAGE ONLY                         822
Ad from 12/13/16                                610
Ad from 11/3/16                                 489
Display Ad created 3/11/15                      392
{KeyWord:Google Brand Items}                    251
{KeyWord:Google Merchandise}                    155
Ad from 11/7/16                                 123
Google Merchandise                               87
First Full Auto Template Test Ad                 87
20% discount                                     75
{KeyWord:Google Branded Gear}                    67
{KeyWord:Looking for Google Bags?}               65
Swag with Google Logos                           64
Display Ad created 11/17/14                      50
{KeyWord:Want Google Stickers?}                  42
JD_5a_v1                                         41
{KeyWord:Goo

##### ``` trafficSource['adwordsClickInfo_adNetworkType']```

In [11]:
t['adwordsClickInfo_adNetworkType'].value_counts()

Google Search      21453
Search partners        7
Name: adwordsClickInfo_adNetworkType, dtype: int64

##### ``` trafficSource['adwordsClickInfo_criteriaParameters']```

In [12]:
checkEmpty = t[t['adwordsClickInfo_criteriaParameters'] != 'not available in demo dataset']
len(checkEmpty)

0

In [13]:
dropList.append('adwordsClickInfo_criteriaParameters')

##### ``` trafficSource['adwordsClickInfo_gclId']```

In [14]:
t['adwordsClickInfo_gclId'].value_counts()

Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3ZdwYO8GTsjiBFd5MPHStZa9y_9NCrI8X97oaAglc8P8HAQ      70
Cj0KEQjw1ee_BRD3hK6x993YzeoBEiQA5RH_BEA562M9tvl_mtnAFvtDnDqOQRp1RvxMMgwjcX1LAfwaAj4o8P8HAQ      41
CJH1vbf94M8CFUElgQodyakHgQ                                                                      29
Cj0KEQiAw_DEBRChnYiQ_562gsEBEiQA4LcssmB_RWgvpPnltzlzj5rGwqx5lk87wC5CjfcqzneNZewaAiAp8P8HAQ      27
CjwKEAiAj7TCBRCp2Z22ue-zrj4SJACG7SBEJui6ggr6ocA-eDC2-lX7W1m5IA1c_qNbzwZVTqUanxoCb5rw_wcB        24
CN_u9PaVhdACFcNahgodTCQAjw                                                                      22
CjwKEAiAxKrFBRDm25f60OegtwwSJABgEC-Z0_DLPcXHm1ZTqlR1YWeWXU875yaqwupt7pGMgFEZThoCeEzw_wcB        21
CjwKEAiAvs7CBRC24rao6bGCoiASJABaCt5DtalFxcoSsvr2E2adUhx6z6OE0KAuVtqKzl-BcVN1-hoCNlrw_wcB        20
CNHp7Nf2ytMCFVlWDQod_IoL5A                                                                      20
CJ6xtee6j9ACFQqDfgod8TkDNw                                                                      18
CjwKEAjw38

In [15]:
dropList.append('adwordsClickInfo_gclId')

##### ``` trafficSource['adwordsClickInfo_isVideoAd']```

Sólo Nans y Falses, drop?

In [16]:
t[t['adwordsClickInfo_isVideoAd'] != False]['adwordsClickInfo_isVideoAd'].isna().value_counts()

True    882193
Name: adwordsClickInfo_isVideoAd, dtype: int64

In [17]:
t[t['adwordsClickInfo_isVideoAd'] == False]['adwordsClickInfo_isVideoAd'].count()

21460

In [18]:
dropList.append('adwordsClickInfo_isVideoAd')

##### ``` trafficSource['adwordsClickInfo_page']```

there might be info here, find out whats the meaning and maybe keep it for seeing correlation with purchases

In [19]:
t['adwordsClickInfo_page'].value_counts()

1     21362
2        73
3        10
5         7
7         3
9         2
4         2
14        1
Name: adwordsClickInfo_page, dtype: int64

##### ``` trafficSource['adwordsClickInfo_slot']```

Creo que es el lugar donde se pone tu anuncio en una busqueta (top si arriba, RHS en los laterales, nan si no viene de un anuncio o no se tiene la info)

In [20]:
t['adwordsClickInfo_slot'].value_counts()

Top    20956
RHS      504
Name: adwordsClickInfo_slot, dtype: int64

##### ``` trafficSource['campaign']```

¿? Bool var isCampaign

In [21]:
t['campaign'].value_counts()

(not set)                                          865347
Data Share Promo                                    16403
AW - Dynamic Search Ads Whole Site                  14244
AW - Accessories                                     7070
test-liyuhz                                           392
AW - Electronics                                       96
Retail (DO NOT EDIT owners nophakun and tianyu)        50
AW - Apparel                                           46
All Products                                            4
Data Share                                              1
Name: campaign, dtype: int64

##### ``` trafficSource['campaignCode']```

In [22]:
dropList.append('campaignCode')

##### ``` trafficSource['isTrueDirect']```

In [23]:
# t['isTrueDirect']

##### ``` trafficSource['keyword']```

check, but drop most likely

In [24]:
t['keyword']

0          (not provided)
1          (not provided)
2          (not provided)
3         google + online
4          (not provided)
5          (not provided)
6          (not provided)
7          (not provided)
8          (not provided)
9          (not provided)
10         (not provided)
11         (not provided)
12         (not provided)
13         (not provided)
14         (not provided)
15                    NaN
16         (not provided)
17         (not provided)
18         (not provided)
19         (not provided)
20         (not provided)
21                    NaN
22         (not provided)
23         (not provided)
24         (not provided)
25         (not provided)
26         (not provided)
27         (not provided)
28                    NaN
29         (not provided)
               ...       
903623                NaN
903624                NaN
903625                NaN
903626                NaN
903627                NaN
903628                NaN
903629                NaN
903630      

##### ``` trafficSource['Medium']```

- merge (not set) and (none)
- cpc -> Cost Per Click 
- affiliate -> Affiliate marketing is a type of performance-based marketing in which a business rewards one or more affiliates for each visitor or customer brought by the affiliate's own marketing efforts.
- cpm -> Cost per impression


¿cpc vienen de search y cpm de webs? ¿Se juntan? ¿Importan?

In [25]:
t['medium'].value_counts()

organic      381561
referral     330955
(none)       143026
cpc           25326
affiliate     16403
cpm            6262
(not set)       120
Name: medium, dtype: int64

##### ``` trafficSource['referralPath']```

In [26]:
dropList.append('referralPath')

##### ``` trafficSource['source']```

¿? regex for the main domain (last whatevs.com), and drop less than 1000

In [27]:
t['source'].value_counts()

google                                               400788
youtube.com                                          212602
(direct)                                             143028
mall.googleplex.com                                   66416
Partners                                              16411
analytics.google.com                                  16172
dfa                                                    5686
google.com                                             4669
m.facebook.com                                         3365
baidu                                                  3356
sites.google.com                                       2983
facebook.com                                           2296
siliconvalley.about.com                                2097
reddit.com                                             2022
qiita.com                                              1813
quora.com                                              1546
bing                                    

***

In [28]:
df['trafficSource'] = t.drop(dropList, axis=1)